In [ ]:
import os 
import urllib 
import shutil
import azureml

from azureml.core import Experiment
from azureml.core import Workspace, Run
from azureml.core import Environment, ScriptRunConfig

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.core.dataset import Dataset

from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

In [ ]:
ws = Workspace.from_config()
dataset = ws.datasets["Refactored Classification Dataset"]
mount_ctx = dataset.as_mount()  
# dataset = Dataset.get_by_name(ws, name='Refactored Classification Dataset')
# dataset.download(target_path='.', overwrite=False)
cluster_name = "gpu-cluster"
compute_target = ComputeTarget(workspace=ws, name=cluster_name)
print('Found existing compute target')

In [ ]:
newenv = Environment.get(workspace=ws, name="Custom-threshold-testing-tf24")

In [ ]:
arguments = args = ['--data-folder', mount_ctx,
                    '--batch-size', 64]

src = ScriptRunConfig(source_directory="./thresholding",
                      script='softmax_calibration.py',
                      arguments=args,
                      compute_target=compute_target,
                      environment=newenv)

In [ ]:
run = Experiment(workspace=ws, name='softmax_calibration').submit(src)
run.wait_for_completion(show_output=True)